In [ ]:
import numpy as np
import numpy.linalg as la
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import matplotlib.pyplot as plt

In [ ]:
# Problem set-up from the multigrid tutorial

a = np.array([[1., 1.],
              [1., 1000.]])
c = np.array([[1000., 1.],
              [1., 1.]])
b = np.array([[0., 2.],
              [0., 0.]])

N = 50
n = N*N
h = (1./(N+1))
ih = 1./(h**2)
i2h = 1./(2*h**2)

A = sp.lil_matrix(((N+2)**2, (N+2)**2)) 
xx = np.zeros((N+2)**2)
yy = np.zeros((N+2)**2)
interior_nodes = set()

for y in range(N+2):
    for x in range(N+2):
        i = y*(N+2) + x
        i_l = y*(N+2) + (x - 1)
        i_r = y*(N+2) + (x + 1)
        i_d = (y-1)*(N+2) + x
        i_u = (y+1)*(N+2) + x
        i_ul = (y+1)*(N+2) + (x-1)
        i_dr = (y-1)*(N+2) + (x+1)
        co_i = np.array([1 if x > (N+2)//2 else 0, 0 if y > (N+2)//2 else 1])
        a_i = a[co_i[1], co_i[0]]
        b_i = b[co_i[1], co_i[0]]
        c_i = c[co_i[1], co_i[0]]
        
        xx[i] = x
        yy[i] = y
        
        if y==0 or y==N+1 or x==0 or x==N+1:
            A[i,i] = 1.
        else:
            interior_nodes.add(i)
            # u_xx
            A[i, i_l] = ih * -a_i
            A[i, i] = ih * -2 * -a_i
            A[i, i_r] = ih * -a_i
            # u_yy
            A[i, i_u] = ih * -c_i
            A[i, i] += ih * -2 * -c_i
            A[i, i_d] = ih * -c_i
            # u_xy
            A[i, i_ul] = i2h * -1 * b_i
            A[i, i_u] += i2h * b_i
            A[i, i_l] += i2h * b_i
            A[i, i] += i2h * -2 * b_i
            A[i, i_r] += i2h * b_i
            A[i, i_d] += i2h * b_i
            A[i, i_dr] = i2h * -1 * b_i

interior_nodes = np.array(list(interior_nodes))
R = sp.eye((N+2)**2).tocsc()[:, interior_nodes]
A = A.tocsr()

In [ ]:
# Lower and strict upper parts of the system for Gauss-Seidel
L = sp.tril(A).tocsr()
U = sp.triu(A, k=1).tocsr()

In [ ]:
# Run 8 iterations of Gauss-Seidel relaxation
def gs(b, x):
    return spla.spsolve_triangular(L, b - U@x)

x = np.random.uniform(size=A.shape[0])
x /= la.norm(x)
b = np.zeros(A.shape[0])

for i in range(8):
    x = gs(b, x)

In [ ]:
# Display the error
plt.figure()
plt.imshow(np.abs(x).reshape((N+2,N+2)))

from mpl_toolkits import mplot3d
fig = plt.figure(figsize=(10,8))
ax = plt.axes(projection='3d')
poly = ax.plot_surface(xx.reshape((N+2,N+2)), yy.reshape((N+2,N+2)), np.abs(x).reshape((N+2,N+2)), cmap='viridis')

In [ ]:
# Conversion to a mesh
verts = np.column_stack((xx, yy, (x / np.max(x)) * 0.8 * 50))

# Insert a quad connecting each DOF
polys = []
for y in range(N+1):
    for x in range(N+1):
        i = y*(N+2) + x
        i_r = y*(N+2) + (x + 1)
        i_u = (y+1)*(N+2) + x
        i_ur = (y+1)*(N+2) + (x + 1)
        polys.append((i, i_r, i_ur, i_u))
        
# Append a bottom face
i_bl = 0
i_br = N+1
i_tl = (N+2)*(N+1)
i_tr = (N+2)**2 - 1
polys.append((i_bl, i_br, i_tr, i_tl))

# And save to an obj file
with open('amg_smooth_model.obj', 'w') as f:
    for vert in verts:
        f.write(f'v {vert[0]} {vert[1]} {vert[2]}\n')
    for poly in polys:
        f.write(f'f {poly[0]+1} {poly[1]+1} {poly[2]+1} {poly[3]+1}\n')